# Web Scraping for Economic Indicators from World Bank

In [5]:
import requests
import pandas as pd


base_url = "https://api.worldbank.org/v2/country"

country_code = "PER"  #PER in this case is PERU, this can change depending on the country user is looking to search 

indicator_codes = [
    'NY.GDP.MKTP.KD.ZG',
    'NV.IND.TOTL.ZS',
    'per_lm_alllm.adq_pop_tot',
    'per_lm_alllm.ben_q1_tot',
    'SL.AGR.0714.ZS',
    'SL.MNF.0714.ZS',
    'SL.SRV.0714.ZS',
    'per_lm_alllm.cov_pop_tot',
    'per_lm_alllm.cov_q2_tot',
    'per_lm_alllm.cov_q3_tot',
    'per_lm_alllm.cov_q4_tot',
    'per_lm_alllm.cov_q1_tot',
    'per_lm_alllm.cov_q5_tot',
    'SL.AGR.EMPL.ZS',
    'SL.IND.EMPL.ZS',
    'SL.SRV.EMPL.ZS',
    'SL.UEM.ADVN.ZS',
    'SL.UEM.BASC.ZS',
    'SL.UEM.INTM.ZS',
    'FR.INR.DPST',
    'GC.XPN.INTP.ZS',
    'GC.XPN.INTP.RV.ZS',
    'GC.XPN.INTP.CN',
    'FR.INR.LNDP'
    ]    #The Economic Series Code represents for economical metrics that users are looking for

format_type = "json"

start_year = 2010

all_data = []

for indicator_code in indicator_codes:
    params = {
        "format": format_type,
        "per_page": 100, 
        "page": 1  }
    
    url = f"{base_url}/{country_code}/indicator/{indicator_code}"
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if len(data) > 1:
            records = data[1]
            for record in records:
                if record.get("value") is not None:  # Filter out missing values
                    year = int(record["date"])
                    if year >= start_year:  # Filter by year
                        all_data.append({
                            "Indicator": record["indicator"]["value"],
                            "Year": year,
                            "Value": record["value"]
                        })
    else:
        print(f"Failed to fetch data for indicator {indicator_code}. Status: {response.status_code}")

df = pd.DataFrame(all_data)

pivot_df = df.pivot(index="Indicator", columns="Year", values="Value")

pivot_df = pivot_df.sort_index(axis=1)

pivot_df.to_csv("Economic Indicators.csv")
